In [1]:
import tensorflow as tf

In [2]:
def resnet50(x, num_class=1000, is_training=True):
    in_layer = x.get_shape()[-1]  # 获取输入的层数 
        
    W_conv1 = tf.Variable(tf.truncated_normal(shape= [7, 7, in_layer, 64], stddev = 0.1))
    conv_1 = tf.nn.conv2d(x, filters=W_conv1, strides=[1, 2, 2, 1], padding='same') # 112x112x64
    nb_1 = tf.layers.batch_normalization(conv_1, axis=3, is_training=is_training) 
    relu_1 = tf.nn.relu(nb_1)
    
    maxpool_1 = tf.nn.max_pool(relu_1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME') # 56x56x64
    
    block1 = block(maxpool_1, out_layer=256, repeat=3, stride=1, is_trainning=is_trainning, scope='block1')
    block2 = block(block1, out_layer=512, repeat=4, stride=2, is_trainning=is_trainning, scope='block2')
    block3 = block(block2, out_layer=1024, repeat=6, stride=2, is_trainning=is_trainning, scope='block3')
    block4 = block(block3, out_layer=2048, repeat=3, stride=2, is_trainning=is_trainning, scope='block4')
    
    avgpool_1 = tf.nn.avg_pool(block4, ksize=[1, 7, 7, 1], strides=[1, 7, 7, 1], padding='SAME' )
    
    squeeze_1 = tf.squeeze(avgpool_1, [1, 2], name='SpatialSqueeze')
    
    W = tf.Variable(tf.truncated_normal(shape = [2048, num_class], stddev = 0.1))
    b = tf.Variable(tf.constant(0.1, shape = [num_class]))
    mul = tf.matmul(x, W) + b
    
    prediction = tf.nn.softmax(mul)
    
    return prediction
print('OK')

OK


In [3]:

def block(x, out_layer, repeat, stride=2, is_trainning=True, scope='block'):
    with tf.variable_scope(scope):
        start_layer = out_layer //4 # RestNet50每一层都是开始输入层数是输出层数的4倍
        out = bottleneck(x, start_layer=start_layer, out_layer=out_layer, stride=stride, is_training=is_training, scope='bottlencek1')
        for i in range(1, repeat):
            out = bottleneck(out, start_layer=start_layer, out_layer=out_layer, stride=1, is_training=is_training, scope=('bottlencek1%ld'%(i+1)))
        return out
print('OK')

OK


In [4]:
import tensorflow as tf
"""
x: 输入值
start_layer : 开始层数
out_layer: 输出层数
stride: 步长
is_training：批量褚思华是否训练
"""

def bottleneck(x, start_layer, out_layer, stride = None, is_training=True, scope='bottleneck'):
    in_layer = x.shape[-1]  # 获取输入的层数
    if stride is None: 
        """
        由于RestNet每个代码块只有两种情况 
        第一种：同层之间(那么x图层的大小和输出的就一样的) (这种情况步长=1)
        第二种：跨层，那么输出x图层大小就等于输入的一般（这种情况下）
        """ 
        stride = 1 if in_layer==out_layer else 2  
    with tf.variable_scope(scope):
        W_conv1 = tf.Variable(tf.truncated_normal(shape= [1, 1, in_layer, start_layer], stddev = 0.1))
        conv_1 = tf.nn.conv2d(x, filter=W_conv1, strides=[1, stride, stride, 1], padding='SAME')
        nb_1 = tf.layers.batch_normalization(conv_1, axis=3, training=is_training)
        relu_1 = tf.nn.relu(conv_1)
        
        W_conv2 = tf.Variable(tf.truncated_normal(shape= [3, 3, start_layer, start_layer], stddev = 0.1))
        conv_2 = tf.nn.conv2d(relu_1, filter=W_conv2, strides=[1, 1, 1, 1], padding='SAME')
        nb_2 = tf.layers.batch_normalization(conv_2, axis=3, training=is_training)
        relu_2 = tf.nn.relu(nb_2)
        
        W_conv3 = tf.Variable(tf.truncated_normal(shape= [3, 3, start_layer, out_layer], stddev = 0.1))
        conv_3 = tf.nn.conv2d(relu_2, filter=W_conv3, strides=[1, 1, 1, 1], padding='SAME')
        nb_3 = tf.layers.batch_normalization(conv_3, axis=3, training=is_training)
    
        if in_layer != out_layer:
            W_short1 = tf.Variable(tf.truncated_normal(shape= [1, 1, in_layer, out_layer], stddev = 0.1))
            conv_4 = tf.nn.conv2d(x, filter=W_short1, strides=[1, stride, stride, 1], padding='SAME')
            short_cut = tf.layers.batch_normalization(conv_4, axis=3, training=is_training)
        else:
            short_cut = x
        return tf.nn.relu(short_cut + nb_3) 
print('OK')

OK


In [5]:
import numpy as np
import tensorflow as tf

x = np.random.random([32, 224, 224, 3]) 
x = x.astype(np.float32)
# predition = resnet50(x)
# predition
# bottleneck(x, 3, 64)
block(x, 64, 3)
# nb_1 = tf.layers.batch_normalization(x, training=True)

# type(x[1][1][1][1])

# x.shape[-1]

NameError: name 'is_training' is not defined